# Notes
- If meet id inclues a `/`, it is probably wrong.  Still don't know why, but we won't use Meet_ID anyways.
- Kyle has TWO athlete id's, 7109122 and 6459186.  Could be a problem.

In [32]:
import pandas as pd
import numpy as np
import utils
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)

In [24]:
mdf = pd.read_csv('M_athlete_results.csv')
fdf = pd.read_csv('F_athlete_results.csv')

In [25]:
# String marks to float times in seconds
mdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(mdf['Event'], mdf['Mark'])]
fdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(fdf['Event'], fdf['Mark'])]

In [26]:
# String events to float distance in meters
mdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in mdf['Event']]
fdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in fdf['Event']]

In [27]:
csm_m = mdf.loc[(mdf['School'] == 'COLORADO MINES') & (mdf['Season'] == 'Outdoor') & (mdf['Event'] == '5000')]
csm_f = fdf.loc[(fdf['School'] == 'COLORADO MINES') & (fdf['Season'] == 'Outdoor') & (fdf['Event'] == '5000')]

In [28]:
xcnats_m = mdf.loc[mdf['Meet_Name'] == 'NCAA Division II Cross Country Championships'].sort_values(['Place', 'Mark'])
xcnats_f = fdf.loc[fdf['Meet_Name'] == 'NCAA Division II Cross Country Championships'].sort_values(['Place', 'Mark'])

In [29]:
f5 = fdf.loc[(fdf['Season'] == 'Outdoor') & (fdf['Distance'] == 5000.0)]
m5 = mdf.loc[(mdf['Season'] == 'Outdoor') & (mdf['Distance'] == 5000.0)]

In [33]:
def prior_to_xc(gender, allresults=False, nats=True):
    if gender == 'M':
        DF = mdf
        xcnats = xcnats_m
    elif gender == 'F':
        DF = fdf
        xcnats = xcnats_f
    # PRs before a specified year
    xcnats_dic = {}
    for year in range(2012, 2020):
        df1 = DF.loc[((DF['Year'] <= year) & (DF['Season'] != 'Cross Country') & (DF['Distance'] > 0)) | ((DF['Year'] < year) & (DF['Season'] == 'Cross Country'))]
        for s in ['Outdoor', 'Indoor']:
            records = utils.get_records()[s]
            for e, m in records.items():
                df1 = df1.loc[(df1['Season'] != s) | (df1['Event'] != e) | ((df1['Season'] == s) & (df1['Event'] == e) & (df1['Time'] >= utils.Time(m)))]
        df2 = xcnats.loc[xcnats['Year'] == year]
        if allresults:
            df3 = df1    # all results rather than just PRs
        else:
            df3 = df1.sort_values('Time').drop_duplicates(subset=['Athlete ID', 'Distance', 'Season'])    # Season specific PRs?
        if nats:
            df = df3.join(df2.set_index('Athlete ID'), on='Athlete ID', rsuffix='NATS')
        else:
            df = df3
            df = df[
                [
                    'Name',
                    'Athlete ID',
                    'Academic_Year',
                    'School', 
                    'Conference',
                    'Meet_Name',
                    'Year',
                    'Season',
                    'Place',
                    'Time',
                    'Distance',
                    'YearNATS',
                    'DistanceNATS',
                    'PlaceNATS',
                    'TimeNATS'
                ]
            ]
            df = df.dropna(subset=['DistanceNATS'])
        xcnats_dic[year] = df
    return xcnats_dic

In [34]:
resvar = True
natsvar = True
xcnats_dic_m = prior_to_xc('M', resvar, natsvar)

AttributeError: module 'utils' has no attribute 'get_records'

In [35]:
if resvar:
    assert len(mdf.loc[(mdf['Name']=='KALE ADAMS') & (mdf['Season']=='Cross Country') & (mdf['Year'] < 2019)]) == len(xcnats_dic_m[2019].loc[(xcnats_dic_m[2019]['Name']=='KALE ADAMS') & (xcnats_dic_m[2019]['Season']=='Cross Country')])

NameError: name 'xcnats_dic_m' is not defined

In [ ]:
del m5
del f5
del csm_f
del csm_m
del xcnats_m
del xcnats_f
del mdf
del fdf

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import sklearn as sk

In [ ]:
df = pd.DataFrame()
for y, d in xcnats_dic_m.items():
    df = pd.concat([df, d])
df['Time_since_PR'] = df['YearNATS']-df['Year']

In [ ]:
for s, dummy in zip(['Cross Country', 'Indoor', 'Outdoor', 'Out of Season'], [.0, 1., 2., 3.]):
    df['Season'][df['Season']==s] = dummy

In [ ]:
def declare(target, features):
    assert type(target) == str
    assert type(features) == list
    targetcol = target
    featurecols = features
    allcols = [f for f in featurecols]
    allcols.append(targetcol)
    return targetcol, featurecols, allcols

In [ ]:
target, feature, allcolumns = declare('PlaceNATS', ['Distance', 'Time'])

In [ ]:
df = df[allcols]
df.shape

In [ ]:
df = df.dropna()
df.shape

In [ ]:
X = df[featurecols]
y = df[targetcol]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X.to_numpy(dtype=np.float64), y.to_numpy(dtype=np.float64), test_size=0.33, random_state=42
)

In [ ]:
X_train.shape